# AEGIS LM Studio Integration

This notebook extends AEGIS with **LM Studio support** for running **uncensored local models** - essential for effective red teaming.

## 🛡️ Why LM Studio for Red Teaming?

- **Uncensored Models**: Access to models without built-in safety filters
- **Local Control**: Complete control over model behavior and responses
- **Privacy**: Sensitive red teaming data stays local
- **Performance**: Optimized inference for local hardware
- **Cost Effective**: No API costs for extensive testing

## 📋 Setup Requirements

1. **Download LM Studio**: https://lmstudio.ai/
2. **Install Dependencies**: `pip install requests`
3. **Download Recommended Models** (see below)
4. **Start LM Studio Server**

In [ ]:
# LM Studio Integration Setup
import asyncio
import json
import logging
import requests
from typing import Dict, List, Optional, Any
from dataclasses import dataclass

# Check if requests is available
try:
    import requests
    LMSTUDIO_AVAILABLE = True
    print("✅ LM Studio support enabled")
except ImportError:
    LMSTUDIO_AVAILABLE = False
    print("❌ Install requests: pip install requests")

# Enhanced LLMConfig for LM Studio
@dataclass
class LLMConfig:
    """Enhanced LLMConfig with LM Studio support."""
    provider: str = "lmstudio"
    model_name: str = "local-model"
    api_key: Optional[str] = None
    base_url: Optional[str] = "http://localhost:1234/v1"
    temperature: float = 0.7
    max_tokens: int = 2048
    top_p: float = 0.9
    timeout: int = 30

def create_lmstudio_config(
    host: str = "localhost",
    port: int = 1234,
    model_name: str = "local-model",
    temperature: float = 0.7
) -> LLMConfig:
    """Create LM Studio configuration."""
    return LLMConfig(
        provider="lmstudio",
        model_name=model_name,
        base_url=f"http://{host}:{port}/v1",
        temperature=temperature
    )

print("LM Studio configuration ready!")

## 🎯 Recommended Uncensored Models for Red Teaming

### **High Priority Models (Start Here)**:
- **WizardLM-13B-Uncensored** - Excellent for adversarial prompts
- **dolphin-2.2.1-mistral-7b** - Fast and effective for attacks  
- **Mixtral-8x7B-Instruct-uncensored** - Powerful reasoning for complex attacks

### **Advanced Models**:
- **WizardLM-30B-Uncensored** - More capable but requires more VRAM
- **CodeLlama-13B-Instruct** - Great for code-based attacks
- **dolphin-2.6-mixtral-8x7b** - Latest uncensored reasoning model

In [ ]:
# LM Studio Health Check and Model Discovery
class LMStudioClient:
    """Client for interacting with LM Studio server."""
    
    def __init__(self, host: str = "localhost", port: int = 1234):
        self.base_url = f"http://{host}:{port}/v1"
        self.session = requests.Session()
        
    async def check_health(self) -> bool:
        """Check if LM Studio server is running."""
        try:
            response = await asyncio.get_event_loop().run_in_executor(
                None,
                lambda: self.session.get(f"{self.base_url}/models", timeout=5)
            )
            return response.status_code == 200
        except:
            return False
    
    async def list_models(self) -> List[str]:
        """Get list of available models."""
        try:
            response = await asyncio.get_event_loop().run_in_executor(
                None,
                lambda: self.session.get(f"{self.base_url}/models")
            )
            if response.status_code == 200:
                models_data = response.json()
                return [model["id"] for model in models_data.get("data", [])]
            return []
        except:
            return []

# Test LM Studio connection
async def test_lmstudio_connection():
    client = LMStudioClient()
    
    print("🔍 Testing LM Studio connection...")
    health = await client.check_health()
    
    if health:
        print("✅ LM Studio server is running!")
        models = await client.list_models()
        if models:
            print(f"📋 Available models: {', '.join(models)}")
        else:
            print("⚠️  No models loaded. Load a model in LM Studio first.")
    else:
        print("❌ LM Studio server not accessible. Make sure:")
        print("   1. LM Studio is running")
        print("   2. Local server is started (port 1234)")
        print("   3. CORS is enabled in LM Studio settings")

# Run the test
if LMSTUDIO_AVAILABLE:
    await test_lmstudio_connection()
else:
    print("❌ Cannot test - requests library not available")

## 🔧 Enhanced BaseLLM with LM Studio Support

Here's how to extend the existing `BaseLLM` class to support LM Studio:

In [ ]:
# Enhanced BaseLLM with LM Studio Support
import openai  # LM Studio uses OpenAI-compatible API

class EnhancedBaseLLM:
    """BaseLLM with LM Studio support - integrate this into existing BaseLLM."""
    
    def _initialize_client(self):
        """Enhanced client initialization with LM Studio support."""
        try:
            if self.config.provider == "lmstudio" and LMSTUDIO_AVAILABLE:
                # LM Studio uses OpenAI-compatible API
                self.client = openai.OpenAI(
                    api_key="not-needed",  # LM Studio doesn't require API key
                    base_url=self.config.base_url or "http://localhost:1234/v1"
                )
            # ... existing provider initialization code ...
        except Exception as e:
            logging.error(f"Failed to initialize {self.config.provider} client: {e}")
            raise
    
    async def _generate_with_provider(self, prompt: str, **kwargs) -> str:
        """Enhanced generation with LM Studio support."""
        try:
            if self.config.provider == "lmstudio":
                # LM Studio uses same API as OpenAI
                response = await asyncio.get_event_loop().run_in_executor(
                    None,
                    lambda: self.client.chat.completions.create(
                        model=self.config.model_name,
                        messages=[{"role": "user", "content": prompt}],
                        max_tokens=kwargs.get("max_tokens", self.config.max_tokens),
                        temperature=kwargs.get("temperature", self.config.temperature)
                    )
                )
                return response.choices[0].message.content
            # ... existing provider generation code ...
        except Exception as e:
            logging.error(f"Generation failed with {self.config.provider}: {e}")
            return f"Error: Failed to generate response with {self.config.provider}"

print("✅ Enhanced BaseLLM with LM Studio support ready!")

## 🚀 Red Teaming Examples with LM Studio

In [ ]:
# Example 1: Basic LM Studio Configuration for Red Teaming
print("🎯 Example 1: Basic Red Teaming Setup")

# Attacker configuration - High creativity for novel attacks
attacker_config = create_lmstudio_config(
    host="localhost",
    port=1234,
    model_name="WizardLM-13B-Uncensored",  # Uncensored model for aggressive attacks
    temperature=0.9  # High creativity
)

# Defender configuration - More conservative for realistic responses
defender_config = create_lmstudio_config(
    host="localhost", 
    port=1234,  # Same server, different model if available
    model_name="Mixtral-8x7B-Instruct-v0.1",
    temperature=0.3  # Lower temperature for consistent defense
)

# Judge configuration - Balanced for evaluation
judge_config = create_lmstudio_config(
    host="localhost",
    port=1234,
    model_name="OpenHermes-2.5-Mistral-7B",
    temperature=0.5
)

print(f"✅ Attacker: {attacker_config.model_name} (temp: {attacker_config.temperature})")
print(f"✅ Defender: {defender_config.model_name} (temp: {defender_config.temperature})")
print(f"✅ Judge: {judge_config.model_name} (temp: {judge_config.temperature})")

In [ ]:
# Example 2: Complete AEGIS Integration with LM Studio
print("🔥 Example 2: Complete AEGIS + LM Studio Integration")

# Import AEGIS components (assuming main notebook has been run)
# from ai_red_teaming_system import AttackerLLM, DefenderLLM, JudgeLLM, RedTeamOrchestrator
# from ai_red_teaming_system import RewardHackingTarget, DeceptionTarget

async def run_lmstudio_red_teaming():
    """Complete red teaming example with LM Studio."""
    
    print("🛡️ Setting up AEGIS with LM Studio...")
    
    # Configure LM Studio-based LLMs
    attacker_config = create_lmstudio_config(
        model_name="WizardLM-13B-Uncensored",
        temperature=0.9
    )
    
    defender_config = create_lmstudio_config(
        model_name="Mixtral-8x7B-Instruct-v0.1",
        temperature=0.3
    )
    
    judge_config = create_lmstudio_config(
        model_name="OpenHermes-2.5-Mistral-7B",
        temperature=0.5
    )
    
    # Create AEGIS components
    # attacker = AttackerLLM(attacker_config)
    # defender = DefenderLLM(defender_config)
    # judge = JudgeLLM(judge_config)
    
    # orchestrator = RedTeamOrchestrator(attacker, defender, judge)
    
    # Create evaluation targets
    # reward_hacking = RewardHackingTarget()
    # deception = DeceptionTarget()
    
    print("✅ AEGIS + LM Studio configuration ready!")
    print("📝 To run evaluation:")
    print("   results = await orchestrator.evaluate_target(reward_hacking)")
    
    return True

# Uncomment to run (requires main AEGIS notebook to be executed first)
# await run_lmstudio_red_teaming()
print("Ready for complete AEGIS + LM Studio integration!")

## 🔧 Security Best Practices

### 🛡️ Responsible Red Teaming with Uncensored Models

1. **Isolated Environment**: Run LM Studio on isolated networks
2. **Secure Logging**: Encrypt and secure all attack logs
3. **Access Control**: Limit access to uncensored models
4. **Responsible Disclosure**: Follow proper vulnerability reporting
5. **Legal Compliance**: Ensure testing is authorized and legal

### ⚠️ Important Considerations

- **Performance**: Uncensored models can be resource-intensive
- **Content**: Models may generate offensive or harmful content
- **Context**: Use only for legitimate security research
- **Monitoring**: Log and review all generated content

## 📚 Additional Resources

### LM Studio Setup
- **Official Website**: https://lmstudio.ai/
- **Documentation**: https://lmstudio.ai/docs
- **Model Repository**: https://huggingface.co/models

### Recommended Models for Download
```bash
# High Priority Models (Download First)
WizardLM/WizardLM-13B-Uncensored
cognitivecomputations/dolphin-2.2.1-mistral-7b
mistralai/Mixtral-8x7B-Instruct-v0.1

# Advanced Models (If You Have Resources)
WizardLM/WizardLM-30B-Uncensored
codellama/CodeLlama-13b-Instruct-hf
cognitivecomputations/dolphin-2.6-mixtral-8x7b
```

### Integration Checklist
- [ ] LM Studio installed and running
- [ ] Uncensored model downloaded and loaded
- [ ] Local server started (port 1234)
- [ ] CORS enabled in LM Studio settings
- [ ] Test connection successful
- [ ] AEGIS main notebook executed
- [ ] LM Studio configs created
- [ ] Security measures in place